In [1]:
import nibabel as nib

In [2]:
import math

In [3]:
import os

In [4]:
import datetime

In [5]:
import heapq

In [6]:
import random

In [7]:
import numpy as np

In [8]:
import pandas as pd

In [9]:
def create_Kernel_Matrix(window_size):
    return np.ones([window_size,window_size,window_size])

In [10]:
#image: the image reference
#index: tuple of x,y,z index
#cluster_ID: cluster ID of the voxel at the specified index
#W - the parameter W
#MinDensity - the parameter MinDensity
#K_W - the kernel matrix
# cluster label -> 0 -> unclassified points; -1 for noise; -2 for border points; cluster_ID for core points
#seeds -> probable -> list of tuples of coordinate indices

def expand_cluster(image,image_size,index,cluster_ID,W,MinDensity,K_W,window_size):
    #print("inside EC")
    seeds = retrieve_neighbours(image,image_size,index,window_size) #get a list of neighbour coordinates
    density = density_function(image,image_size,index,window_size,K_W)
    
    
    #print("density",str(density))
    
#     print("**seeds begin***")
#     print(seeds)
#     print("*** seeds end ***")
    
    
    if density < MinDensity:
        #print(str(index[0])+" "+str(index[1])+" "+str(index[2]))
        
        voxel_labels[index[0]][index[1]][index[2]] = -1 #noise
        #print("Finish EC")
        return False #return expansion failure
    else:
        
        #for all xj in seeds
        #coords is a tuple containing index/voxel coordinates of Xj
        for coords in seeds:
#             #neighbours -> list containing neighbours of Xj
            neighbours = retrieve_neighbours(image,image_size,coords,window_size)
            density_Xj = density_function(image,image_size,coords,window_size,K_W)
            
            if density_Xj >= MinDensity:
                voxel_labels[coords[0]][coords[1]][coords[2]] = cluster_ID #core points
                
                for coords_nbr in neighbours:
                    if voxel_labels[coords_nbr[0]][coords_nbr[1]][coords_nbr[2]] == 0:
                        seeds.append(coords_nbr)
                for coords_nbr in neighbours:
                    label = voxel_labels[coords_nbr[0]][coords_nbr[1]][coords_nbr[2]]
                    if label == 0 or label == -1:
                        voxel_labels[coords_nbr[0]][coords_nbr[1]][coords_nbr[2]] = -2 #labelled as border points
        
        #print("Finish EC")    
        return True #return expansion success

In [11]:
def find_neighbour_distance(index,neighbour_index):
    val = (index[0]-neighbour_index[0])**2 + (index[1]-neighbour_index[1])**2 + (index[2]-neighbour_index[2])**2
    distance = math.sqrt(val)
    return distance

In [12]:
#reserved for step 3 of XMT DBSCAN

def find_border_points(image,image_size,window_size,voxel_labels):
    border_index_array = np.where(voxel_labels == -2)
    x_idx = border_index_array[0]
    y_idx = border_index_array[1]
    z_idx = border_index_array[2]
    
    for count in range(0,len(x_idx)):
        index = (x_idx[count],y_idx[count],z_idx[count])
        core_label_list = []
        core_distance_list = []
        core_nbr_list = []
        coords_list = retrieve_neighbours(image,image_size,index,window_size)
        
        #remove non core neighbours
        for coords in coords_list:
            label = voxel_labels[coords[0]][coords[1]][coords[2]]
            
            if label > 0:
                core_label_list.append(label) # implies the point is a core point
                core_nbr_list.append(coords) #store label and coordinate in core neighbours list
         
        for nbr_count in range(0,len(core_nbr_list)):
                nbr_index = core_nbr_list[nbr_count]
                distance = find_neighbour_distance(index,nbr_index)
                core_distance_list.append(distance)
                
        smallest_idx = heapq.nsmallest(1, range(len(core_distance_list)), key=core_distance_list.__getitem__) #for multiple matches, chooses first index
        nearest_core_idx = smallest_idx[0]
        nearest_core_nbr = core_nbr_list[nearest_core_idx]
        nearest_cluster_ID = voxel_labels[nearest_core_nbr[0]][nearest_core_nbr[1]][nearest_core_nbr[2]]
        voxel_labels[x_idx[count]][y_idx[count]][z_idx[count]] = nearest_cluster_ID
        

In [13]:
def retrieve_neighbours(image,image_size,index,window_size,neighbour_values=False):
    #print("inside RN")
    nbr_coords,nbr_values = create_half_window(image,index,image_size,window_size)
      
    #print("Fin RN")
    if neighbour_values == True:
        return nbr_coords, nbr_values
    else:
        return nbr_coords

In [14]:
def density_function(image,image_size,index,window_size,K_W):
    #print("inside DF")
    nbr_coords,nbr_values = retrieve_neighbours(image,image_size,index,window_size,neighbour_values=True)
    
    product = np.multiply(nbr_values,K_W)
    count_nnz = np.count_nonzero(product) 
    density = 0
    summation = np.sum(product) 
        
    if count_nnz > 0:
        density = summation/count_nnz
    

    #print("Prod type",type(product))
    #print("prod shape",str(product.shape))
    
#     print("sum",str(temp_sum))
#     print("nnz",str(count_nnz))
    
    #print("Finished DF")
    return density

In [15]:
def create_half_window(image,index,image_size,window_size):
    #print("inside CHW")
    xaxis_lower_limit = 0
    xaxis_higher_limit = image_size[0] - 1
    
    yaxis_lower_limit = 0
    yaxis_higher_limit = image_size[1] - 1
    
    zaxis_lower_limit = 0
    zaxis_higher_limit = image_size[2] - 1
    
    nbr_window_length = math.floor(window_size/2)
    
    xaxis_low = index[0] - nbr_window_length
    xaxis_high = index[0] + nbr_window_length
    
    yaxis_low = index[1] - nbr_window_length
    yaxis_high = index[1] + nbr_window_length
    
    zaxis_low = index[2] - nbr_window_length
    zaxis_high = index[2] + nbr_window_length
    
    half_window_coords = list()
    half_window_values = np.zeros([window_size,window_size,window_size])
    
    xaxis_left_difference = 0
    xaxis_right_difference = 0
    yaxis_top_difference = 0
    yaxis_bottom_difference = 0
    zaxis_anterior_difference = 0
    zaxis_posterior_difference = 0
    
    window_xlow = 0
    window_xhigh = window_size - 1
    window_ylow = 0
    window_yhigh = window_size - 1
    window_zlow = 0
    window_zhigh = window_size - 1
    
    
    if xaxis_low < xaxis_lower_limit:
        xaxis_left_difference = xaxis_lower_limit - xaxis_low
        xaxis_low = xaxis_lower_limit
                
    if xaxis_high > xaxis_higher_limit:
        xaxis_right_difference = xaxis_high - xaxis_higher_limit 
        xaxis_high = xaxis_higher_limit
        
    if yaxis_low < yaxis_lower_limit:
        yaxis_top_difference = yaxis_lower_limit - yaxis_low
        yaxis_low = yaxis_lower_limit
          
    if yaxis_high > yaxis_higher_limit:
        yaxis_bottom_difference = yaxis_high - yaxis_higher_limit
        yaxis_high = yaxis_higher_limit
        
    if zaxis_low < zaxis_lower_limit:
        zaxis_anterior_difference = zaxis_lower_limit - zaxis_low
        zaxis_low = zaxis_lower_limit
        
    if zaxis_high > zaxis_higher_limit:
        zaxis_posterior_difference = zaxis_high - zaxis_higher_limit
        zaxis_high = zaxis_higher_limit
        
    window_ylow = window_ylow + yaxis_top_difference
    window_yhigh = window_yhigh - yaxis_bottom_difference
    
    window_xlow = window_xlow + xaxis_left_difference
    window_xhigh = window_xhigh - xaxis_right_difference
    
    window_zlow = window_zlow + zaxis_anterior_difference
    window_zhigh = window_zhigh - zaxis_posterior_difference
    
    
#     print("x ld",str(xaxis_left_difference))
#     print("x rd",str(xaxis_right_difference))
#     print("y td",str(yaxis_top_difference))
#     print("y bd",str(yaxis_bottom_difference))
    
#     print("win xl",str(window_xlow))
#     print("win xh",str(window_xhigh))
#     print("win yl",str(window_ylow))
#     print("win yh",str(window_yhigh))
    
    
    window_row = -1
    window_col = -1
    
    window_slice = window_zlow
    for z_img in range(zaxis_low,zaxis_high + 1):
        window_row = window_ylow
        
        for y_img in range(yaxis_low,yaxis_high + 1):
            window_col = window_xlow
            
            for x_img in range(xaxis_low,xaxis_high + 1):
                value = image[x_img][y_img][z_img]
                half_window_values[window_row][window_col][window_slice] = value
                
                if not(x_img == index[0] and y_img == index[1] and z_img == index[2]):
                    half_window_coords.append((x_img,y_img,z_img))
                
#                 print("---------------------------------------")
#                 print(str(x_img)+" "+str(y_img)+" "+str(z_img))
#                 print(str(window_row)+" "+str(window_col)+" "+str(window_slice))  
#                 print("---------------------------------------")
                
                window_col = window_col + 1
            window_row = window_row + 1
        window_slice = window_slice + 1
    
    half_window_values[nbr_window_length][nbr_window_length][nbr_window_length] = 0
       
    #print("finished CHW")
    return half_window_coords, half_window_values
    
    
    
    

Set the parameters W and MinDensity

Run XMT-DBSCAN

In [16]:
x_coords = []
for x in range(10):
    x_coords.append(random.randint(0,120))

In [17]:
y_coords = []
for y in range(10):
    y_coords.append(random.randint(0,144))

In [18]:
z_coords = []
for z in range(10):
    z_coords.append(random.randint(0,120))

In [19]:
voxel_coordinates = []
for x in x_coords:
    for y in y_coords:
        for z in z_coords:
            new_index = (x,y,z)
            voxel_coordinates.append(new_index)

In [20]:
len(voxel_coordinates)

1000

Load image

In [45]:
image = nib.load(r'D:\Fresh Alzheimers Work - smw images\CN vs AD\GM\Training\masked_smw_images\mask_smwc151_ADNI_002_S_0619_MR_MP-RAGE__br_raw_20060601215738863_1_S15147_I16392.nii')
imageData = image.get_fdata()
imageSize = imageData.shape

In [46]:
MinDensity = 0.5
W = 3
windowSize = 2*W+1
K_W = create_Kernel_Matrix(windowSize)
voxel_labels = np.zeros(imageSize) #get unclassified labels for voxels

In [47]:
print(datetime.datetime.now())

cluster_ID = 1

for index in voxel_coordinates:
    #print(index)      
    
    result = expand_cluster(imageData,imageSize,index,cluster_ID,W,MinDensity,K_W,windowSize)
    if result == True:
        cluster_ID = cluster_ID + 1  


find_border_points(imageData,imageSize,windowSize,voxel_labels)
            
print(datetime.datetime.now())

2019-02-20 19:15:28.957599
2019-02-20 19:17:38.865115


In [48]:
cluster_ID

2

In [49]:
image_affine = image.affine

In [50]:
voxel_label_image = nib.Nifti1Image(voxel_labels, affine=image_affine)

In [51]:
nib.save(voxel_label_image,'D:\\images\\voxel_labels_smwc151_W3_0.5M.nii')

In [160]:
file_list = os.listdir(r'D:\Fresh Alzheimers Work - smw images\CN vs AD\GM\Training\masked_smw_images')

In [161]:
path = 'D:\\Fresh Alzheimers Work - smw images\\CN vs AD\\GM\\Training\\masked_smw_images\\'

In [162]:
MinDensity = 0.9

In [163]:
W = 3
windowSize = 2*W+1
K_W = create_Kernel_Matrix(windowSize)

In [165]:
for file in file_list:
    file_path = path+file
    image = nib.load(file_path)
    imageData = image.get_fdata()
    imageSize = imageData.shape
    voxel_labels = np.zeros(imageSize) #get unclassified labels for voxels
    
    print("Image", file)
    print(datetime.datetime.now())

    cluster_ID = 1

    for index in voxel_coordinates:
    #print(index)      
    
        result,voxel_labels = expand_cluster(imageData,imageSize,index,cluster_ID,W,MinDensity,K_W,windowSize,voxel_labels)
        if result == True:
            cluster_ID = cluster_ID + 1  


    find_border_points(imageData,imageSize,windowSize,voxel_labels)
    image_affine = image.affine
    voxel_label_image = nib.Nifti1Image(voxel_labels, affine=image_affine)
    image_name = 'D:\\images\\'+file
    nib.save(voxel_label_image,image_name)
    print(datetime.datetime.now())
    print("*****************************************************************")
    
    

Image mask_smwc110_ADNI_009_S_0751_MR_MP_RAGE__br_raw_20060801143332101_5_S17487_I20013.nii
2019-02-20 18:59:42.713062
2019-02-20 18:59:44.446438
*****************************************************************
Image mask_smwc111_ADNI_009_S_0842_MR_MP_RAGE__br_raw_20060927083107095_1_S18870_I24339.nii
2019-02-20 18:59:44.462067
2019-02-20 18:59:46.221778
*****************************************************************
Image mask_smwc112_ADNI_009_S_0862_MR_MP_RAGE__br_raw_20060927102700674_1_S19358_I25128.nii
2019-02-20 18:59:46.230771
2019-02-20 18:59:47.959663
*****************************************************************
Image mask_smwc113_ADNI_014_S_0519_MR_MP-RAGE__br_raw_20060515195749046_1_S14488_I15323.nii
2019-02-20 18:59:47.975289
2019-02-20 18:59:49.691167
*****************************************************************
Image mask_smwc114_ADNI_014_S_0520_MR_MP-RAGE__br_raw_20060515171041368_1_S14474_I15299.nii
2019-02-20 18:59:49.706767
2019-02-20 18:59:51.437748
******

2019-02-20 19:00:52.769648
*****************************************************************
Image mask_smwc156_ADNI_005_S_0221_MR_MP-RAGE__br_raw_20061006154159590_1_S19845_I25942.nii
2019-02-20 19:00:52.769648
2019-02-20 19:00:54.487416
*****************************************************************
Image mask_smwc157_ADNI_005_S_0814_MR_MP-RAGE__br_raw_20060830124436257_1_S18390_I23573.nii
2019-02-20 19:00:54.503041
2019-02-20 19:00:56.262074
*****************************************************************
Image mask_smwc158_ADNI_005_S_0929_MR_MP-RAGE__br_raw_20061003144555138_1_S19669_I25645.nii
2019-02-20 19:00:56.277696
2019-02-20 19:00:58.006571
*****************************************************************
Image mask_smwc159_ADNI_005_S_1341_MR_MP-RAGE__br_raw_20070307150517807_1_S27673_I43188.nii
2019-02-20 19:00:58.022195
2019-02-20 19:00:59.752127
*****************************************************************
Image mask_smwc15_ADNI_006_S_0731_MR_MP-RAGE__br_raw_200608

2019-02-20 19:02:01.946220
*****************************************************************
Image mask_smwc19_ADNI_007_S_1222_MR_MP-RAGE__br_raw_20070123224843055_158_S25401_I37270.nii
2019-02-20 19:02:01.961847
2019-02-20 19:02:03.650329
*****************************************************************


In [62]:
# for x in range(0,120):
#     for y in range(0,144):
#         for z in range(0,120):
#             print(voxel_labels[x][y][z])

In [28]:
# def second_largest(numbers,index=False):
#     count = 0
#     m1 = m2 = float('-inf')
#     for x in numbers:
#         count += 1
#         if x > m2:
#             if x >= m1:
#                 m1, m2 = x, m1            
#             else:
#                 m2 = x
#     return m2 if count >= 2 else None

In [31]:
# def second_smallest(numbers):
#     count = 0
#     m1 = m2 = float('inf')
#     for x in numbers:
#         count += 1
#         if x < m2:
#             if x <= m1:
#                 m1, m2 = x, m1            
#             else:
#                 m2 = x
            
#     return m2 if count >= 2 else None

In [1]:
# fr = [8,4,1,1,1,12]
# s = heapq.nsmallest(1, range(len(fr)), key=fr.__getitem__)

# index = (60,40,20)
#coords,values = create_half_window(imageData,(60,44,20),imageSize,windowSize)
# x1 = np.zeros([2,2])
# x1[0][0] = 1
# x1[0][1] = 2
# x1[1][0] = 3
# x1[1][1] = 4

# x2 = np.zeros([2,2,2])
# x2[0][0][0] = 2
# x2[0][1][0] = 4
# x2[1][0][0] = 6
# x2[1][1][0] = 8


# x_img = 61
# y_img = 40
# z_img = 20

# if not(x_img == index[0] and y_img == index[1] and z_img == index[2]):
#         print('hello world')

#  row_start = 0
#     row_end = window_size - 1
    
#     col_start = 0
#     col_end = window_size - 1
    
#     slice_start = 0
#     slice_end = window_size-1
    
#     xaxis_left_difference = 0
#     xaxis_right_difference = 0
    
#     yaxis_top_difference = 0
#     yaxis_bottom_difference = 0
    
#     zaxis_anterior_difference = 0
#     zaxis_posterior_difference = 0
    
   
#     if xaxis_low < xaxis_lower_limit:
#         xaxis_left_difference = xaxis_lower_limit - xaxis_low
#         xaxis_low = xaxis_low + xaxis_left_difference
                
#     if xaxis_high > xaxis_higher_limit:
#         xaxis_right_difference = xaxis_high - xaxis_higher_limit 
#         xaxis_high = xaxis_high - xaxis_right_difference
        
#     if yaxis_low < yaxis_lower_limit:
#         print("yl low")
#         yaxis_bottom_difference = yaxis_lower_limit - yaxis_low
#         yaxis_low = yaxis_low + yaxis_bottom_difference
#         print(yaxis_low)
        
#     if yaxis_high > yaxis_higher_limit:
#         yaxis_top_difference = yaxis_high - yaxis_higher_limit
#         yaxis_high = yaxis_high - yaxis_top_difference
        
#     if zaxis_low < zaxis_lower_limit:
#         print("zl low")
#         zaxis_anterior_difference = zaxis_lower_limit - zaxis_low
#         zaxis_low = zaxis_low + zaxis_anterior_difference
        
#     if zaxis_high > zaxis_higher_limit:
#         zaxis_posterior_difference = zaxis_high - zaxis_higher_limit
#         zaxis_high = zaxis_high - zaxis_posterior_difference

        
#     row_start = row_start + yaxis_top_difference
#     row_end = row_end - yaxis_bottom_difference
    
#     col_start = col_start + xaxis_left_difference
#     col_end = col_end - xaxis_right_difference
    
    
    
#     print("row start",str(row_start))
#     print("row end",str(row_end))
    
#     print("col start",str(col_start))
#     print("col end",str(col_end))
    
#     slice_start = slice_start + zaxis_anterior_difference
#     slice_end = slice_end - zaxis_posterior_difference
    
#     print("y ax hi",str(yaxis_high))
#     print("x ax lo",str(xaxis_low))
    
#     x_img = -1
#     y_img = -1
#     z_img = -1
    
#     z_img = zaxis_low
#     for z_idx in range(slice_start,slice_end + 1):
#         y_img = yaxis_high
        
#         for y_idx in range(col_end,col_start - 1,-1):
#             x_img = xaxis_low
            
#             for x_idx in range(row_start,row_end + 1):
#                 #half_window_values[x_idx][y_idx][z_idx] = image[x_img][y_img][z_img]
#                 #half_window_coords[x_idx][y_idx][z_idx] = str(x_img) +" "+str(y_img)+" "+str(z_img)
#                 print(str(x_img) +" "+str(y_img)+" "+str(z_img))
#                 x_img = x_img + 1
            
#             y_img = y_img - 1
        
#         z_img = z_img + 1

In [6]:
#index - array of x,y,z indices
#voxel_labels -> stores cluster labels of individual voxels

#cluster label = -1 for noise
#0 for border points

# def label_Voxels(index, cluster_label, voxel_labels):
#     voxel_labels[index[0]][index[1]][index[2]] = cluster_label
    
    
    
# #,all_pixels = False
# #     if all_pixels == True:
# #         for zaxis in img_size[2]:
# #             for yaxis in img_size[1]:
# #                 for xaxis in img_size[0]:
# #                     voxel_labels[xaxis][yaxis][zaxis] = -1


#      if xaxis_low < xaxis_lower_limit:
#         for idx in range(xaxis_low,xaxis_lower_limit):
#             xaxis_low_temp.add(idx)          
#         xaxis_low = xaxis_lower_limit
                
#     elif xaxis_high > xaxis_higher_limit:
#         for idx in range(xaxis_high,xaxis_higher_limit,-1):
#             xaxis_high_temp.add(idx)          
#         xaxis_high = xaxis_higher_limit
        
#     elif yaxis_low < yaxis_lower_limit:
#         for idx in range(yaxis_low,yaxis_lower_limit):
#             yaxis_low_temp.add(idx)          
#         yaxis_low = yaxis_lower_limit
        
#     elif yaxis_high > yaxis_higher_limit:
#         for idx in range(yaxis_high,yaxis_higher_limit,-1):
#             yaxis_high_temp.add(idx)          
#         yaxis_high = yaxis_higher_limit
        
#     elif zaxis_low < zaxis_lower_limit:
#         for idx in range(zaxis_low,zaxis_lower_limit):
#             zaxis_low_temp.add(idx)    
#         zaxis_low = zaxis_lower_limit
        
#     elif zaxis_high > zaxis_higher_limit:
#         for idx in range(zaxis_high,zaxis_higher_limit,-1):
#             zaxis_high_temp.add(idx)          
#         zaxis_high = zaxis_higher_limit
        
    
    